In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

Importing the dataset given by the Dog Breed Identification 

In [ ]:
breeds = pd.read_csv("labels.csv")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

One-hot encoding for strings

In [ ]:
encoder = LabelBinarizer()
one_hot = encoder.fit_transform(breeds['breed'])
one_hot.shape

In [ ]:
y_train = np.asarray(one_hot)

In [ ]:
labels = np.array(y_train, np.uint8) #data, datatype

In [ ]:
x_train = []
y_train = Y_train

In [ ]:
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions
batch_size = 32
def train_gen():
    while True:
        for i in range(0,len(train_imgs),batch_size):
            if i+batch_size < len(train_imgs):
                batch_imgs = train_imgs[i:i+batch_size]
                y_batch = labels[i:i+batch_size]
            else:
                batch_imgs = train_imgs[i:]
                y_batch = labels[i:]
                x_batch = []
            for img in batch_imgs:
                Image = image.load_img(\"train/\" + img + \".jpg\", target_size=(299, 299))
                data = image.img_to_array(Image)
                data = preprocess_input(data)
                x_batch.append(data)
            yield((np.array(x_batch,np.float32),y_batch))

In [ ]:
from keras import Model
from keras.layers import Input, Dense
from keras.applications.inception_resnet_v2 import InceptionResNetV2

backbone = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(299,299,3))
outputs = Dense(120, activation='softmax')(backbone.output)
model = Model(inputs=backbone.input, outputs=outputs)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = [\"accuracy\"])

In [ ]:
model.fit_generator(train_gen(), steps_per_epoch=np.ceil(len(train_imgs)/batch_size), epochs=10, verbose=1)

In [ ]:
model.save_weights('model-v1.h5')

In [ ]:
import pickle

In [ ]:
with open('encoder.pkl','wb') as f:
    pickle.dump(encoder,f)

In [5]:
from tensorflow.contrib import lite
converter = lite.TFLiteConverter.from_keras_model_file( 'model.h5' ) # Your model's name
model = converter.convert()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 898 variables.
INFO:tensorflow:Converted 898 variables to const ops.


In [6]:
file = open( 'model.tflite' , 'wb' ) 
file.write( model )

217937980